# 데이터 불러오기

In [1]:
import glob
import os
import re
import tensorflow as tf

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'
txt_list=glob.glob(txt_file_path)

raw_corpus=[]

for txt_file in txt_list:
    with open(txt_file,'r') as f:
        raw=f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:",len(raw_corpus))
print('example:\n',raw_corpus[:3])

데이터 크기: 187088
example:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


# 데이터 전처리

In [2]:
def preprocess_sentence(sentence):
    import re
    sentence = sentence.lower().strip() #1
    sentence = re.sub(r"(\[.*\])", '', sentence) #2
    sentence = re.sub(r"([?.!])", r" \1 ", sentence) # 3
    sentence = re.sub(r"('[s|t])", r" \1", sentence) # 4
    sentence = re.sub(r'[" "]+', " ", sentence) # 5
    sentence = re.sub(r"[^a-zA-Z?.!']+", " ", sentence) # 6
    sentence = re.sub(r'[" "]+', " ", sentence) #5
    sentence = sentence.strip() # 7
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
2. 특수문자 양쪽에 공백을 넣고
3. 여러개의 공백은 하나의 공백으로 바꿉니다
4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
5. 다시 양쪽 공백을 지웁니다
6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다


In [3]:
corpus = []

for sentence in raw_corpus:
   
    if len(sentence) == 0: continue
    
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        

corpus[:10]

["<start> now i've heard there was a secret chord <end>",
 '<start> that david played and it pleased the lord <end>',
 "<start> but you don 't really care for music do you ? <end>",
 '<start> it goes like this <end>',
 '<start> the fourth the fifth <end>',
 '<start> the minor fall the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

* 빈 문장 제외하고 정제된 문장을 corpus에 넣는다

In [4]:
def tokenize(corpus):
    
    tokenizer=tf.keras.preprocessing.text.Tokenizer(
    num_words=12000,
    filters='',
    oov_token="<unk>")
    
    
    tokenizer.fit_on_texts(corpus)
    tensor=tokenizer.texts_to_sequences(corpus)
    #입력데이터의 시퀀스 길이를  padding으로 맞춰줌, 최대 토큰 길이는 15
    tensor=tf.keras.preprocessing.sequence.pad_sequences(tensor,maxlen=15,padding='post')
    
        
    print(tensor,tokenizer)
    return tensor,tokenizer

tensor,tokenizer=tokenize(corpus)
tensor.shape  

[[   2   49  147 ...    0    0    0]
 [   2   15 2677 ...    0    0    0]
 [   2   34    6 ...    3    0    0]
 ...
 [ 127   21    8 ...    9 1027    3]
 [8413    5   36 ... 1293  645    3]
 [   2    6   33 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f357c2acdf0>


(175986, 15)

* corpus를 텐서로 변환
* 토큰의 개수를 15로 제한시켜줌 (maxlen=15)

# 데이터 분리(훈련/검증/테스트)

In [5]:
from sklearn.model_selection import train_test_split

src_input = tensor[:, :-1]
tgt_input = tensor[:, 1:]  

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,
                                                          tgt_input,
                                                          test_size=0.2,
                                                          random_state=32)

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (140788, 14)
Target Train: (140788, 14)


In [7]:
BUFFER_SIZE = len(src_input) 
BATCH_SIZE = 128       
steps_per_epoch = BUFFER_SIZE // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1

dataset=tf.data.Dataset.from_tensor_slices((src_input,tgt_input))
dataset=dataset.shuffle(BUFFER_SIZE)
dataset=dataset.batch(BATCH_SIZE,drop_remainder=True)
dataset

<BatchDataset shapes: ((128, 14), (128, 14)), types: (tf.int32, tf.int32)>

# 모델 설계

In [8]:
class TextGenerator(tf.keras.Model):
    def __init__(self,vocab_size,embedding_size,hidden_size):
        super().__init__()
        
        self.embedding=tf.keras.layers.Embedding(vocab_size,embedding_size)
        self.rnn1=tf.keras.layers.LSTM(hidden_size,return_sequences=True)
        self.rnn2=tf.keras.layers.LSTM(hidden_size,return_sequences=True)
        self.linear=tf.keras.layers.Dense(vocab_size)
        
    def call(self,x):
        out=self.embedding(x)
        out=self.rnn1(out)
        out=self.rnn2(out)
        out=self.linear(out)
        
        return out

# 모델 생성

In [9]:
vocab_size = tokenizer.num_words + 1 
embedding_size=256
hidden_size=2048
model=TextGenerator(vocab_size,embedding_size,hidden_size)

* vocab_size는 총 단어 개수에 pad:0 이 포함된 크기
* embedding_size는 단어가 추상적으로 표현되는 크기
* hidden_size 는 모델에 얼마나 많은 일꾼을 두느냐(많으면배가 산으로 갈 수도 있음)

In [10]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(128, 14, 12001), dtype=float32, numpy=
array([[[-1.44935155e-04, -2.05210614e-04,  6.44014290e-05, ...,
          5.18250636e-05, -4.36777518e-05,  1.60979194e-04],
        [-2.25001946e-04, -1.82580607e-05,  3.18568840e-04, ...,
          1.71400330e-04, -1.93362343e-04, -5.52994970e-05],
        [-2.18223577e-04, -1.38959702e-04,  3.72762734e-04, ...,
         -7.35447538e-05, -1.90920531e-04, -2.88842071e-04],
        ...,
        [-7.24552548e-04,  4.37345181e-04,  8.55459541e-04, ...,
         -2.62384856e-04, -6.82816841e-04, -1.03099307e-03],
        [-7.88262987e-04,  4.92267311e-04,  7.69754639e-04, ...,
         -2.02371913e-04, -4.71976382e-04, -8.50314915e-04],
        [-7.16526585e-04,  5.99232619e-04,  9.28453635e-04, ...,
         -1.06380598e-04, -6.30124821e-04, -7.08819716e-04]],

       [[-1.44935155e-04, -2.05210614e-04,  6.44014290e-05, ...,
          5.18250636e-05, -4.36777518e-05,  1.60979194e-04],
        [-2.35080879e-04, -5.90304699e-05, -1

# 모델 학습

In [11]:
optimizer = tf.keras.optimizers.Adam()

loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss,optimizer=optimizer)

model.fit(enc_train, dec_train, validation_data=(enc_val, dec_val),epochs=10, batch_size=128)

Epoch 1/10
1100/1100 [==============================] - 393s 355ms/step - loss: 3.4750 - val_loss: 3.1076
Epoch 2/10
1100/1100 [==============================] - 316s 287ms/step - loss: 2.9198 - val_loss: 2.8331
Epoch 3/10
1100/1100 [==============================] - 316s 287ms/step - loss: 2.6006 - val_loss: 2.6551
Epoch 4/10
1100/1100 [==============================] - 317s 288ms/step - loss: 2.2936 - val_loss: 2.5280
Epoch 5/10
1100/1100 [==============================] - 316s 288ms/step - loss: 2.0039 - val_loss: 2.4356
Epoch 6/10
1100/1100 [==============================] - 317s 288ms/step - loss: 1.7393 - val_loss: 2.3800
Epoch 7/10
1100/1100 [==============================] - 316s 287ms/step - loss: 1.5120 - val_loss: 2.3491
Epoch 8/10
1100/1100 [==============================] - 317s 288ms/step - loss: 1.3257 - val_loss: 2.3461
Epoch 9/10
1100/1100 [==============================] - 316s 288ms/step - loss: 1.1814 - val_loss: 2.3691
Epoch 10/10
1100/1100 [=======================

# 평가하기

In [12]:
def generate_text(model,tokenizer,init_sentence='<start>',max_len=20):
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]
    
    while True:
        predict = model(test_tensor) 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
       
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break
        
    generated = ""
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "
    return generated

In [13]:
generate_text(model, tokenizer, init_sentence="<start> you raise", max_len=20)

'<start> you raise my hand and we kissed in the moonlight <end> '

# 회고

* 토큰 길이 조절: tf.keras.preprocessing.sequence.pad_sequences() 의 maxlen으로 조절할 수 있었다.


* val_loss 를 조절하기위해서 각종 하이퍼파라미터들을 조절했는데 2.3언저리에서 안떨어진다. 더군다나 7epoch 이후에는 훈련모델에 과대적합이
  되는건지 val_loss의 수치가 다시 올라간다. 
  
  
* 그리드서치를 배우고 써먹으려고 시도해봤으나 오류가 자꾸만 뜬다. 보통 그리드서치에 들어가는 모델의 예시로는 의사결정나무,랜덤포레스트 등등 만들어진 모델로 쓰긴하는데... 내가 만든 모델로는 그리드 서치가 안되는건가 의문이 든다.